<a href="https://colab.research.google.com/github/Qwerci/Q-A_1/blob/main/Rag_Q_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets torch faiss-cpu wikipedia-api langchain sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.2 MB/s eta 0:00:00


In [4]:
import wikipediaapi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import torch
import faiss
import numpy as np

In [7]:
def get_wikipedia_content(topic):
    wiki = wikipediaapi.Wikipedia(language='en',user_agent='RagPipeline/1.0 (kwesiqwerci@gmail.com)')
    page = wiki.page(topic)
    return page.text

def preprocess_text(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 50,
        length_function=len,
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [8]:
def create_embeddings(chunks):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings


In [9]:
def create_faiss_index(embeddings):
    embeddings = np.array(embeddings).astype('float32')
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype('float32'))
    return index




In [10]:
def retrieve_relevant_passages(query_embedding, index, chunks, k=5):
    query_embedding = np.array(query_embedding).astype('float32').reshape(1, -1)
    distances, indices = index.search(query_embedding.astype('float32'), k)
    return [chunks[i] for i in indices[0]]

In [11]:
def query_understanding(query):
    return query

In [12]:
# Load model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
def answer_generation(query, relevant_passages):
    context = "\n".join(relevant_passages)
    prompt = f"Based on the following context, answer the question: {query}\n\nContext: {context}\n\nAnswer"

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=150, num_return_sequences=1, temperature=0.7)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [14]:
def rag_pipeline(query, embeddings, faiss_index, chunks):
    # Query understanding
    processed_query = query_understanding(query)

    # Create query embedding
    query_embedding = create_embeddings([processed_query])

    # Retrieve relevant passages
    relevant_passages = retrieve_relevant_passages(query_embedding, faiss_index, chunks)

    # Generate answer
    answer = answer_generation(processed_query, relevant_passages)

    return answer

In [16]:
    # Collect and preprocess data
    topic = "Artificial Intelligence"
    print(f"Collecting data on {topic}...")
    raw_text = get_wikipedia_content(topic)
    chunks = preprocess_text(raw_text)

    print(f"Collected amd preprocessed{len(chunks)} chuncks from Wikipedia on {topic}")

    # Create embedding
    print("Creating embeddings...")
    embeddings = create_embeddings(chunks)
    print(f"Created embeddings with shape: {embeddings.shape}")

    # Create FAISS index
    print("Creating FAISS index...")
    faiss_index = create_faiss_index(embeddings)
    print(f"Created FAISS index with {faiss_index.ntotal} vectors")


    # Example usage
    while True:
        query = input("\nEnter your question about Artificial Intelligence (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        print("Generating answer...")
        result = rag_pipeline(query, embeddings, faiss_index, chunks)
        print(f"\nQuery: {query}")
        print(f"\nAnswer: {result}")

    print("Thank you for using the AI Q&A system!")

Collected amd preprocessed263 chuncks from Wikipedia on Artificial Intelligence
Creating embeddings...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Created embeddings with shape: (263, 384)
Creating FAISS index...
Created FAISS index with 263 vectors

Enter your question about Artificial Intelligence (or 'quit' to exit): Who discovered it?
Generating answer...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


...


Query: Are you saying Alan Turing discovered AI in 2012?

Answer: no

Enter your question about Artificial Intelligence (or 'quit' to exit): So who discovered AI and which year was it discovered?
Generating answer...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: So who discovered AI and which year was it discovered?

Answer: 1956

Enter your question about Artificial Intelligence (or 'quit' to exit): quite
Generating answer...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: quite

Answer: a bit

Enter your question about Artificial Intelligence (or 'quit' to exit): quit
Thank you for using the AI Q&A system!
